In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
from typing import Any, Optional, List
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()

from ccxt.base.exchange import Exchange
from threading import Thread, Event, Lock
from queue import Queue
# from multiprocessing import Queue #as Queue

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.series import TimeSeries, Bar, Trade, Quote, OHLCV
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, dt_64
from qubx.impl.ccxt_connector import CCXTConnector, CtrlChannel, BinanceQV

 >  [dev] installed cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



# Common

In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
orderbook_max_levels_limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - orderbook_max_levels_limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=orderbook_max_levels_limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# Listen to data

In [3]:
class RunThread(Thread):
    def __init__(self, channel: CtrlChannel):
        self.result = None
        self.ch = channel
        self.loops = []
        super().__init__()

    def add(self, func, *args, **kwargs):
        self.loops.append(func(self.ch, *args, **kwargs))

    async def run_loop(self):
        self.result = await asyncio.gather(*self.loops)

    def run(self):
        self.ch.control.set()
        asyncio.run(self.run_loop())

    def stop(self):
        self.ch.control.clear()
        self.ch.queue.put((None, None)) # send sentinel


async def listen_to_trades(channel: CtrlChannel, exchange: Exchange, symbols: List[str]):
    method = 'watchTrades'
    if exchange.has[method]:
        # lock = channel.lock
        while channel.control.is_set():
            try:
                # c_time = exchange.iso8601(exchange.milliseconds())
                trades = await exchange.watch_trades_for_symbols(symbols)
                # lock.acquire()
                for trade in trades:
                    # trade = trades[-1]
                    t_ns = trade['timestamp'] * 1_000_000 # this is trade time 
                    s = trade['symbol']
                    info = trade['info']
                    price = trade['price']
                    m = info['m']
                    amnt = trade['amount'] # in base currency
                    tas = Trade(t_ns, price, amnt, int(not m), int(trade['id']))
                    channel.queue.put((s, tas))
                # lock.release()
                # print(trade, flush=True)
                # print(f"\t[{c_time}] {s} : {str(tas)}", flush=True)
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await exchange.close()
                raise e
    else:
        raise Exception(exchange.id + ' ' + method + ' is not supported or not implemented yet')


async def listen_to_orderbook(channel: CtrlChannel, exchange: Exchange, symbols: List[str], orderbook_max_levels_limit=5):
    lock = channel.lock
    while channel.control.is_set():
        try:
            orderbook = await exchange.watch_order_book_for_symbols(symbols, limit=orderbook_max_levels_limit)
            # - check what this time is ?
            t_ns = orderbook['timestamp'] * 1_000_000
            s = orderbook['symbol']
            b, a = orderbook['bids'], orderbook['asks']
            q = Quote(t_ns, b[0][0], a[0][0], b[0][1], a[0][1])
            # lock.acquire()
            channel.queue.put((s, q))
            # lock.release()
            # c_time = exchange.iso8601(exchange.milliseconds())
            # print(f"[{c_time}] {s} : {str(q)}", flush=True)
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e
 

async def receiver(channel: CtrlChannel):
    sers = {}
    print("START LISTENING", flush=True)
    T, Q = 0, 0
    while channel.control.is_set():
        s, data = channel.queue.get()
        if isinstance(data, Quote):
            if not (Q % 100):
                print(s, data, flush=True)
                Q = 0
            Q += 1
        if isinstance(data, Trade):
            if s not in sers:
                sers[s] = TimeSeries(s, '1Min')
            ts = sers[s]
            ts.update(data.time, data.price)
            if not (T % 100):
                print(s, data, flush=True)
                T = 0
            T += 1
        if isinstance(data, Bar):
            # print(f"{s} {pd.Timestamp(data.time, unit='ns')}: {data}", flush=True)
            if s not in sers:
                sers[s] = {}
            sers[s][data.time] = data

    print(f"STOP LISTENING: {str(sers)}", flush=True)
    return sers 

In [4]:
def prefetch_ohlcs(exchange: Exchange, symbol: str, timeframe: str, nbarsback: int):
    assert nbarsback > 1
    start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)
    return ohlcv

async def listen_to_ohlcv(channel: CtrlChannel, exchange: Exchange, 
                          symbol: List[str], timeframe: str, nbarsback: int):
    # - check if we need to load initial 'snapshot'
    if nbarsback > 1:
        ohlcv = await prefetch_ohlcs(exchange, symbol, timeframe, nbarsback)
        for oh in ohlcv:
            b = Bar(oh[0] * 1_000_000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
            channel.queue.put((symbol, b))

    while channel.control.is_set():
        try:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe)
            for oh in ohlcv:
                b = Bar(oh[0] * 1000000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
                channel.queue.put((symbol, b))
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e

In [5]:
symbs = ['1INCHUSDT', 'AGLDUSDT', 'APEUSDT', 'ARBUSDT', 'AUCTIONUSDT', 'BANDUSDT', 'BATUSDT', 'C98USDT', 'COMPUSDT', 'CYBERUSDT',
        'DYDXUSDT', 'ENSUSDT', 'FXSUSDT', 'HFTUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'IOTXUSDT', 'LDOUSDT', 'LQTYUSDT', 'LTCUSDT',
        'MASKUSDT', 'MBOXUSDT', 'MINAUSDT', 'MTLUSDT', 'OGNUSDT', 'OPUSDT', 'PEOPLEUSDT', 'POLSUSDT', 'PYRUSDT', 'QNTUSDT', 'RADUSDT',
        'RDNTUSDT', 'RIFUSDT', 'RLCUSDT', 'SEIUSDT', 'SPELLUSDT', 'SSVUSDT', 'STGUSDT', 'STXUSDT', 'SUIUSDT', 'UMAUSDT', 'WAVESUSDT']

In [6]:
chan = CtrlChannel('ticks')
reader = RunThread(chan)
reader.add(receiver)
reader.start()

START LISTENING


In [7]:
# bu = cxp.binanceusdm()
# bu = cxp.binance()
bu = BinanceQV()
# bu = getattr(cxp, 'binanceqv')()
t1 = RunThread(chan)
for s in symbs:
    t1.add(listen_to_ohlcv, bu, s, '15m', 10)

# t1.add(listen_to_ohlcv, bu, 'BTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'ETHUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'LTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'SOLUSDT', '5m', 10)
# t1.add(listen_to_trades, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.add(listen_to_orderbook, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
t1.start()

In [10]:
reader.stop()
t1.stop()

In [ ]:
print(len(reader.result[0]), len(symbs))
reader.result[0]

# StrategyCtx

In [163]:
from qubx import logger

class ZeroStrat(IStrategy):
    param1: int = 0

    def on_start(self, ctx: StrategyContext):
        logger.info(f"> Started with param1 = {self.param1}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> Optional[List[Signal]]:
        logger.info(f"{event.time} -> {event}")
        return None

In [153]:
cc = CCXTConnector('binance')
cc.time()

numpy.datetime64('2024-03-28T14:23:46.550100000')

In [ ]:
# cc.get_historical_ohlcs('BTCUSDT', '1d', 20)

In [154]:
ctx = StrategyContext(
    ZeroStrat(), dict(param1 = 1), cc, cc, 
    instruments=lookup.instruments[r'BINANCE:(BTC|ETH|LTC)USDT'],
    fees_spec='vip0_usdt', base_currency='USDT',

    md_subscription=dict(type='ohlc', timeframe='1Min', nback=10),
    trigger=dict(type='bar', delay='-1s', timeframe='1Min')
)

2024-03-28 16:23:47.425 [ ℹ️ ] Set param1 -> 1


In [ ]:
ctx.start()

2024-03-28 16:23:48.054 [ ℹ️ ] Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 10} for 
	['BTCUSDT', 'ETHUSDT', 'LTCUSDT'] 
2024-03-28 16:23:48.054 [ ℹ️ ] > Started with param1 = 1
2024-03-28 16:23:48.056 [ ℹ️ ] > Data processor started
2024-03-28 16:23:48.058 [ ℹ️ ] Start processing market data
2024-03-28 16:23:48.092 [ ℹ️ ] > Market data subscribtions started


2024-03-28 16:23:50.575 [ ℹ️ ] 2024-03-28T14:23:50.575336000 -> TriggerEvent(time=numpy.datetime64('2024-03-28T14:23:50.575336000'), type='bar', instrument='BTCUSDT', data={o:70740.010000 | h:70825.980000 | l:70740.000000 | c:70790.000000 | v:7507510.353455})
2024-03-28 16:23:50.576 [ ℹ️ ] 2024-03-28T14:23:50.575336000 -> TriggerEvent(time=numpy.datetime64('2024-03-28T14:23:50.575336000'), type='bar', instrument='BTCUSDT', data={o:70790.000000 | h:70860.000000 | l:70790.000000 | c:70847.760000 | v:2466864.829787})
2024-03-28 16:23:50.576 [ ℹ️ ] 2024-03-28T14:23:50.576543000 -> TriggerEvent(time=numpy.datetime64('2024-03-28T14:23:50.576543000'), type='bar', instrument='BTCUSDT', data={o:70847.770000 | h:70925.190000 | l:70830.000000 | c:70830.010000 | v:1829000.874902})
2024-03-28 16:23:50.577 [ ℹ️ ] 2024-03-28T14:23:50.577543000 -> TriggerEvent(time=numpy.datetime64('2024-03-28T14:23:50.577543000'), type='bar', instrument='BTCUSDT', data={o:70830.010000 | h:70875.870000 | l:70754.20000

In [156]:
ctx.stop()

2024-03-28 16:23:59.748 [ ℹ️ ] Market data processing finished


In [161]:
ctx.ohlc('BTCUSDT', '1Min')

                         open      high       low     close        volume  \
timestamp                                                                   
2024-03-28 14:14:00  70740.01  70825.98  70740.00  70790.00  7.507510e+06   
2024-03-28 14:15:00  70790.00  70860.00  70790.00  70847.76  2.466865e+06   
2024-03-28 14:16:00  70847.77  70925.19  70830.00  70830.01  1.829001e+06   
2024-03-28 14:17:00  70830.01  70875.87  70754.20  70772.23  3.317511e+06   
2024-03-28 14:18:00  70772.23  70830.80  70699.82  70830.79  3.773713e+06   
2024-03-28 14:19:00  70830.80  70980.00  70830.79  70964.00  2.585844e+06   
2024-03-28 14:20:00  70964.01  70964.01  70890.52  70954.96  2.063747e+06   
2024-03-28 14:21:00  70954.96  71000.00  70935.99  70945.99  2.400951e+06   
2024-03-28 14:22:00  70946.00  70988.25  70936.00  70938.01  1.615149e+06   
2024-03-28 14:23:00  70938.01  70982.00  70899.24  70952.02  2.391804e+06   

                     bought_volume  
timestamp                           
2

In [162]:
await cc.exch.close()

In [ ]:
# cc.close()

<hr/>

# Misc

In [105]:
%load_ext autoreload
%autoreload 2

from qubx.utils.time import convert_tf_str_td64
from qubx.core.utils import recognize_time
from qubx.core.strategy import OhlcvsHolder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
holder = OhlcvsHolder('1Min')
holder.get_ohlcv('ETH', '1Min')
holder.get_ohlcv('ETH', '5Min')

Empty DataFrame
Columns: [open, high, low, close, volume, bought_volume]
Index: []

In [137]:
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:00'), 1, 1, 1, 1, 1))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:01'), 1, 4, 1, 3, 2, 1))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:03'), 1, 4, 1, 3, 2, 1))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:03'), 1, 4, 1, 3, 2, 1))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:04'), 1, 4, 1, 3, 2, 1))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:05'), 1, 4, 1, 3, 2, 1))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:05'), 1, 4, 1, 3, 2, 2))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:05'), 1, 4, 1, 3, 2, 11))
holder.update_by_bar('ETH', Bar(recognize_time('2020-01-01 00:05'), 1, 4, 1, 3, 2, 11))
holder.get_ohlcv('ETH', '1Min')

                     open  high  low  close  volume  bought_volume
timestamp                                                         
2020-01-01 00:00:00   1.0   1.0  1.0    1.0     1.0            0.0
2020-01-01 00:01:00   1.0   4.0  1.0    3.0     2.0            1.0
2020-01-01 00:03:00   1.0   4.0  1.0    3.0     2.0            1.0
2020-01-01 00:04:00   1.0   4.0  1.0    3.0     2.0            1.0
2020-01-01 00:05:00   1.0   4.0  1.0    3.0     2.0           11.0

In [138]:
holder.get_ohlcv('ETH', '1Min')

                     open  high  low  close  volume  bought_volume
timestamp                                                         
2020-01-01 00:00:00   1.0   1.0  1.0    1.0     1.0            0.0
2020-01-01 00:01:00   1.0   4.0  1.0    3.0     2.0            1.0
2020-01-01 00:03:00   1.0   4.0  1.0    3.0     2.0            1.0
2020-01-01 00:04:00   1.0   4.0  1.0    3.0     2.0            1.0
2020-01-01 00:05:00   1.0   4.0  1.0    3.0     2.0           11.0

In [129]:
holder.get_ohlcv('ETH', '5Min')

                     open  high  low  close  volume  bought_volume
timestamp                                                         
2020-01-01 00:00:00   1.0   4.0  1.0    3.0     7.0            3.0
2020-01-01 00:05:00   1.0   4.0  1.0    3.0     2.0           11.0

In [ ]:
# OHLCV.test = lambda self: print("TETET")

In [3]:
from qubx.core.series import TimeSeries, Bar, Trade, Quote, OHLCV

In [8]:
ser0 = OHLCV('test', '5Min')
ser0.update_by_bar(recognize_time('2020-01-01 00:00:00').item(), 1, 1, 1, 1, 10, 1)
ser0.update_by_bar(recognize_time('2020-01-01 00:01:00').item(), 2, 2, 2, 2, 1, 2)
ser0.update_by_bar(recognize_time('2020-01-01 00:06:00').item(), 1, 2, 2, 2, 10)

1

In [9]:
ser0

                     open  high  low  close  volume  bought_volume
timestamp                                                         
2020-01-01 00:00:00   1.0   2.0  1.0    2.0    11.0            3.0
2020-01-01 00:05:00   1.0   2.0  2.0    2.0    10.0            0.0